In [3]:
from google.colab.drive import mount
mount("/content/drive",force_remount = True)

Mounted at /content/drive


In [4]:
import os

images_dir = "/content/drive/MyDrive/ITMO/01_BRIGHT_FIELD/original_img"
masks_dir = "/content/drive/MyDrive/ITMO/01_BRIGHT_FIELD/original_msk"

image_count = 0
mask_count = 0

if os.path.exists(images_dir):
    image_count = len([f for f in os.listdir(images_dir) if os.path.isfile(os.path.join(images_dir, f))])

if os.path.exists(masks_dir):
    mask_count = len([f for f in os.listdir(masks_dir) if os.path.isfile(os.path.join(masks_dir, f))])

print(f"Number of files in {images_dir}: {image_count}")
print(f"Number of files in {masks_dir}: {mask_count}")

Number of files in /content/drive/MyDrive/ITMO/01_BRIGHT_FIELD/original_img: 857
Number of files in /content/drive/MyDrive/ITMO/01_BRIGHT_FIELD/original_msk: 857


# Biologically plausible transformations.

In [ ]:
import os
import cv2
import numpy as np
import albumentations as A
from tqdm import tqdm
import random

# --- Mount Google Drive (run only once in Colab) ---
from google.colab.drive import mount
mount("/content/drive", force_remount=True)

# --- Directories ---
image_dir = "/content/drive/MyDrive/ITMO/01_BRIGHT_FIELD/original_img"
mask_dir = "/content/drive/MyDrive/ITMO/01_BRIGHT_FIELD/original_msk"
output_image_dir = "/content/drive/MyDrive/ITMO/01_BRIGHT_FIELD/augmented_img"
output_mask_dir = "/content/drive/MyDrive/ITMO/01_BRIGHT_FIELD/augmented_msk"


os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_mask_dir, exist_ok=True)

# Get image and mask files sorted
image_files = sorted([f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])
mask_files = sorted([f for f in os.listdir(mask_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])

# --- Mask Binarization Function ---
def binarize_mask(mask):
    return (mask > 127).astype(np.uint8) * 255

# --- Primary Transforms (applied to image, and mask if needed) ---
primary_transforms = [
    ("Original", None),  # Include original
    ("Original", None),  # Include original
    ("Original", None),  # Include original

    ("Blur", A.Blur(blur_limit=3, p=1)),
    ("MedianBlur", A.MedianBlur(blur_limit=3, p=1)),
    ("GaussianBlur", A.GaussianBlur(blur_limit=3, p=1)),
    ("HueSaturationValue", A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=1)),
    ("RGBShift", A.RGBShift(r_shift_limit=20, g_shift_limit=20, b_shift_limit=20, p=1)),
    ("ChannelShuffle", A.ChannelShuffle(p=1)),
    ("ColorJitter", A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, p=1)),
    ("ToGray", A.ToGray(p=1)),
    ("InvertImg", A.InvertImg(p=1)),


    ("MotionBlur", A.Compose([
        A.MotionBlur(blur_limit=(3, 7), p=1),
        A.Lambda(mask=binarize_mask)
    ], additional_targets={'mask': 'image'})),

    ("Defocus", A.Compose([
        A.Defocus(radius=(1, 3), p=1),
        A.Lambda(mask=binarize_mask)
    ], additional_targets={'mask': 'image'})),

    ("ZoomBlur", A.Compose([
        A.ZoomBlur(max_factor=1.05, p=1),
        A.Lambda(mask=binarize_mask)
    ], additional_targets={'mask': 'image'})),

    # Noise and color — image only
    ("GaussianNoise", A.GaussNoise(mean=0, std=10.0, p=1)),
    ("ISONoise", A.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.3), p=1)),
    ("LowContrast", A.RandomGamma(gamma_limit=(70, 130), p=1)),
    ("CLAHE", A.CLAHE(clip_limit=1.0, p=1)),
    ("BrightnessContrast", A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=1))
]

# --- Secondary Transforms (always applied to both image and mask) ---
secondary_transforms = [
    ("Rotate15", A.SafeRotate(limit=15, interpolation=cv2.INTER_NEAREST, p=1)),

    ("ElasticDeform", A.ElasticTransform(alpha=30, sigma=5, interpolation=cv2.INTER_NEAREST, p=1)),

    ("GridDistort", A.GridDistortion(num_steps=3, distort_limit=0.1, interpolation=cv2.INTER_NEAREST, p=1)),

    ("MitoticCrop", A.RandomResizedCrop(size=(256, 256), scale=(0.9, 1.0), ratio=(0.95, 1.05),
                                        interpolation=cv2.INTER_NEAREST, p=1)),

    ("DivisionShift", A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=10,
                                         interpolation=cv2.INTER_NEAREST, border_mode=cv2.BORDER_CONSTANT, p=1)),

    ("CrowdingAffine", A.Affine(translate_percent=(0.03, 0.03), scale=(0.9, 1.1),
                                rotate=(-5, 5), shear=(-2, 2),
                                interpolation=cv2.INTER_NEAREST, cval=0, p=1)),

    ("HorizontalFlip", A.HorizontalFlip(p=1)),
    ("VerticalFlip", A.VerticalFlip(p=1)),
    ("RandomRotate90", A.RandomRotate90(p=1)),
    ("Transpose", A.Transpose(p=1)),

    ("PadAndCrop", A.Compose([
        A.PadIfNeeded(min_height=512, min_width=512, border_mode=cv2.BORDER_CONSTANT,
                      image_pad_value=0, mask_pad_value=0),
        A.RandomCrop(height=256, width=256)
    ], additional_targets={'mask': 'image'}))
]

for img_file, mask_file in tqdm(zip(image_files, mask_files), total=len(image_files)):
    img_path = os.path.join(image_dir, img_file)
    mask_path = os.path.join(mask_dir, mask_file)

    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    base_name = os.path.splitext(img_file)[0]

    # Save original
    cv2.imwrite(os.path.join(output_image_dir, f"{base_name}_original.jpg"),
                cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
    cv2.imwrite(os.path.join(output_mask_dir, f"{base_name}_original.jpg"), mask)

    n_augmentations = len(primary_transforms) - 1  # excluding original
    '''
    # Setup plot: 2 columns (image, mask), rows = original + augmentations
    total_rows = n_augmentations + 1
    plt.figure(figsize=(8, 4 * total_rows))

    # Plot original image + mask
    plt.subplot(total_rows, 2, 1)
    plt.imshow(image)
    plt.title("Original Image")
    plt.axis('off')

    plt.subplot(total_rows, 2, 2)
    plt.imshow(mask, cmap='gray')
    plt.title("Original Mask")
    plt.axis('off')
    '''
    # Loop over augmentations

    # Apply augmentations
    for i, (prim_name, prim_transform) in enumerate(primary_transforms, start=1):
        # For the first 3 "Original" transforms, skip primary transform
        if i <= 3:
            transformed_img = image.copy()
            transformed_mask = mask.copy()
        else:
            # Apply primary transform
            if isinstance(prim_transform, A.Compose):
                result = prim_transform(image=image, mask=mask)
                transformed_img = result['image']
                transformed_mask = result['mask']
            else:
                result = prim_transform(image=image)
                transformed_img = result['image']
                transformed_mask = mask.copy()  # Keep original mask if transform is image-only

        # Apply random secondary transform to both image and mask
        sec_name, sec_transform = random.choice(secondary_transforms)
        final_result = sec_transform(image=transformed_img, mask=transformed_mask)
        final_img = final_result['image']
        final_mask = final_result['mask']

        # Save augmented image and mask
        cv2.imwrite(os.path.join(output_image_dir, f"{base_name}_aug_{i-1}.jpg"),
                    cv2.cvtColor(final_img, cv2.COLOR_RGB2BGR))
        cv2.imwrite(os.path.join(output_mask_dir, f"{base_name}_aug_{i-1}.jpg"), final_mask)
'''
        # Plot image and mask side-by-side
        plt.subplot(total_rows, 2, 2*i - 3)
        plt.imshow(final_img)
        plt.title(f"{prim_name} + {sec_name}")
        plt.axis('off')

        plt.subplot(total_rows, 2, 2*i - 2)
        plt.imshow(final_mask, cmap='gray')
        plt.title(f"{prim_name} + {sec_name} (mask)")
        plt.axis('off')

    plt.tight_layout()
    plt.show()
'''
print("Augmentation and display complete!")